In [22]:
import json, os, datetime
from typing import Tuple, Any
import pandas as pd, numpy as np
from scipy import stats
from src.commons.tools import input_path, output_path, check_directories, input_numeric_col

pd.set_option("display.max_columns", None)

check_directories()

cat_cols = ['Desc_Cargo', 'Proyecto', 'genero']


def feature_dane(df: pd.DataFrame):
    total_counting = {
        'TP27_PERSO': 'persons', #número total de personas
        'TVIVIENDA': 'houses', #conteo de viviendas
        'CTNENCUEST': 'surveys', #cantidad de encuestas
        'TP16_HOG': 'homes'
    }
    feature_bars = {
        'TP27_PERSO': [
            'TP51_13_ED', 'TP51SUPERI', 'TP51SECUND', 'TP51PRIMAR', 'TP51_99_ED', 'TP34_6_EDA',
            'TP34_8_EDA', 'TP34_7_EDA', 'TP34_3_EDA', 'TP34_5_EDA', 'TP34_9_EDA', 'TP34_4_EDA',
            'TP34_2_EDA', 'TP34_1_EDA', 'TP32_1_SEX', 'TP32_2_SEX', 'TP51POSTGR'
        ],
        'TVIVIENDA': [
            'TP9_1_USO', 'TP19_INTE1', 'TP19_GAS_1', 'TP19_ACU_1', 'TP19_GAS_9',
            'TP19_EE_E2', 'TP19_EE_E3', 'TP19_EE_E5', 'TP19_EE_E6', 'TP15_1_OCU',
            'TP14_2_TIP', 'TP9_2_USO', 'TP14_6_TIP', 'TP15_2_OCU', 'TP14_4_TIP',
        ],
        'CTNENCUEST': ['TP4_2_NO', 'TP3_2_NO'],
        'TP16_HOG': ['TP27_PERSO']
    }
    featured_dataset, drop_vars = df.copy(), []
    for key, value in feature_bars.items():
        for var in value:
            featured_dataset[f'{total_counting[key]}_{var}'] = featured_dataset[var]/featured_dataset[key]
        drop_vars.extend(value)
    featured_dataset = featured_dataset.drop(np.unique(drop_vars), axis=1)
    return featured_dataset

def get_high_correlated_features(df: pd.DataFrame) -> pd.DataFrame:
    corr_matrix = df.corr().abs()
    corr_matrix = corr_matrix[(corr_matrix>0.8)&(corr_matrix<1)]
    filter_ = corr_matrix.isna().all()
    hyper_correlated = corr_matrix[corr_matrix.columns[~corr_matrix.columns.isin(filter_[filter_].index.tolist())]]
    v1, v2, corr = [], [], []
    for col in hyper_correlated.columns:
        for idx in hyper_correlated.index:
            if not pd.isna(hyper_correlated[col].loc[idx]):
                aux = [col, idx]
                aux.sort()
                v1.append(aux[0])
                v2.append(aux[1])
                corr.append(hyper_correlated[col].loc[idx])
    hyper_correlated = pd.DataFrame({'variable1': v1, 'variable2': v2, 'correlation': corr}).drop_duplicates()
    return hyper_correlated

def years_computing(file_path: str) -> pd.DataFrame:
    desc_cargo_eq = {
        "CONDUCTOR VOLQUETA DAF": "CONDUCTOR DE VOLQUETA DAF",
        "AUXILIAR ADMINISTRATIVA": "AUXILIAR ADMINSTRATIVO",
        "INSPECTOR SST": "INSPECTOR SST I",
        "SOLDADOR ": "SOLDADOR I"
    }
    dataset = pd.read_csv(
        file_path,
        parse_dates=['fecha_nacimiento']
    )
    dataset['Desc_Cargo'] = dataset['Desc_Cargo'].replace(desc_cargo_eq)
    dataset_ = dataset.copy()
    dataset_.insert(10, 'anios', (datetime.datetime.now()-dataset.fecha_nacimiento).dt.days//365.25)
    dataset_ = dataset_.drop('fecha_nacimiento', axis=1)
    dataset_ = dataset_[~(dataset_.causa_retiro=='MUERTE DEL TRABAJADOR')]
    return dataset_

def outliers_remotion(dataset_: pd.DataFrame) -> pd.DataFrame:
    dataset_.loc[dataset_.anios<18, 'anios'] = np.nan
    dataset_.loc[dataset_.anios>60, 'anios'] = np.nan
    dataset_ = input_numeric_col(dataset_, 'anios')
    return dataset_

def get_dummies(dataset_: pd.DataFrame) -> pd.DataFrame:
    cat_dataset = dataset_[cat_cols]
    numeric_data = dataset_[dataset_.columns[~dataset_.columns.isin(cat_cols)]]
    objective_var = numeric_data[['causa_retiro']]
    numeric_data = numeric_data.drop('causa_retiro', axis=1)
    #setting dtypes
    numeric_data = numeric_data.astype({'anios': int})
    dummies = (pd.get_dummies(cat_dataset)*1).drop('genero_F', axis=1)
    dataset_ = dummies.join(numeric_data).join(objective_var)
    #encoding scope variable
    dataset_.loc[dataset_.causa_retiro=='TERMINACION DE CONTRATO', 'causa_retiro'] = 1
    dataset_.loc[dataset_.causa_retiro!=1, 'causa_retiro'] = 0
    return dataset_

def drop_non_variant_cols(dataset_: pd.DataFrame) -> pd.DataFrame:
    #Droping columns with unique values
    no_variation_cols = dataset_.corr().isna().all()
    no_variation_cols = no_variation_cols[no_variation_cols].index.tolist()
    dataset_ = dataset_[dataset_.columns[~dataset_.columns.isin(no_variation_cols)]]
    return dataset_

def droping_irrelevant_variables(dataset_: pd.DataFrame, file_path: str) -> pd.DataFrame:
    #Droping columns with no correlation with objective variable ('causa_retiro')
    corr_matrix = dataset_.corr()
    corr_matrix.to_excel(os.path.join(os.path.dirname(file_path), 'correlation_matrix.xlsx'), index=0)
    relevant_variables = corr_matrix.loc['causa_retiro'].abs()
    relevant_variables = relevant_variables[relevant_variables>0.05].index.tolist()
    dataset_ = dataset_[relevant_variables]
    return dataset_

def droping_redundant_variables(dataset_: pd.DataFrame) -> pd.DataFrame:
    #Droping highly correlated columns
    featured_dataset = feature_dane(dataset_)
    correlated_features = get_high_correlated_features(featured_dataset) #use this to get highly correlated variables
    cols_high_correlated = [
        'Desc_Cargo_AYUDANTE DE OBRA tasa 6.96', 'CTNENCUEST', 'TP16_HOG',
        'TVIVIENDA', 'TP9_2_2_MI', 'TP19_RECB1', 'TP19_INTE2', 'TP19_EE_1',
        'TP19_ALC_1', 'TP19_INTE9', 'TP15_4_OCU', 'TP19_RECB2'
    ]
    featured_dataset = featured_dataset.drop(cols_high_correlated, axis=1)
    scope = featured_dataset.causa_retiro
    featured_dataset = featured_dataset.drop('causa_retiro', axis=1)
    featured_dataset['retiro'] = scope
    return featured_dataset


In [ ]:
def process_prediction_dataset(file_path: str) -> pd.DataFrame:
    set_ = os.path.basename(file_path).split('_')[0]
    dataset_ = years_computing(file_path)
    dataset_ = outliers_remotion(dataset_)
    dataset_ = get_dummies(dataset_)
    if set_=='train':
        dataset_ = drop_non_variant_cols(dataset_)
        dataset_ = droping_irrelevant_variables(dataset_, file_path)
        featured_dataset = droping_redundant_variables(dataset_)
    else:
        featured_dataset = feature_dane(dataset_)
        scope = featured_dataset.causa_retiro
        featured_dataset = featured_dataset.drop('causa_retiro', axis=1)
        featured_dataset['retiro'] = scope
        schema = os.path.join('..', 'input', 'prediction-data-mining-schema.json')
        with open(schema) as f:
            schema = json.loads(f.read())
        featured_dataset = featured_dataset[schema['schema']]
    featured_dataset.to_csv(os.path.join(os.path.dirname(file_path), 'non_correlated_dataset.csv'), index=0)
    return featured_dataset

In [46]:
file_path = os.path.join(output_path, 'predictive_mining', 'train_set', 'train_without_featuring.csv')
featured_train_dataset = process_prediction_dataset(file_path)

In [47]:
file_path = os.path.join(output_path, 'predictive_mining', 'deploy_set', 'deploy_without_featuring.csv')
featured_train_dataset = process_prediction_dataset(file_path)

KeyError: "['Desc_Cargo_AUXILIAR AMBIENTAL', 'Desc_Cargo_CADENERO I', 'Desc_Cargo_INSPECTOR SST I', 'Desc_Cargo_PERFORISTA -LANZADOR', 'Desc_Cargo_SOLDADOR I', 'retiro'] not in index"

In [274]:
hyper_correlated

,variable1,variable2,correlation
0,Desc_Cargo_AYUDANTE DE OBRA tasa 6.96,salario_mes,0.886780
2,CTNENCUEST,TVIVIENDA,0.997743
3,CTNENCUEST,TP16_HOG,0.993754
5,TP16_HOG,TVIVIENDA,0.995672


In [251]:
hyper_correlated[hyper_correlated.variable1=='TP15_4_OCU']

,variable1,variable2,correlation
39,TP15_4_OCU,TP3_2_NO,0.869212
72,TP15_4_OCU,TP4_2_NO,0.869212
107,TP15_4_OCU,TP9_1_USO,0.873774
144,TP15_4_OCU,TVIVIENDA,0.872268
249,TP15_4_OCU,TP16_HOG,0.840200
250,TP15_4_OCU,TP19_EE_1,0.843869
251,TP15_4_OCU,TP19_ACU_1,0.841286
252,TP15_4_OCU,TP19_ALC_1,0.841503
253,TP15_4_OCU,TP19_GAS_1,0.846400
254,TP15_4_OCU,TP19_RECB1,0.843602


In [275]:
hyper_correlated.groupby('variable1')[['variable1']].value_counts().sort_values(ascending=False)

variable1
CTNENCUEST                               2
Desc_Cargo_AYUDANTE DE OBRA tasa 6.96    1
TP16_HOG                                 1
Name: count, dtype: int64